In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score
from transformers import BlipProcessor, BlipModel
import torch.nn as nn


Mounted at /content/drive


In [2]:
dataPath = "/content/drive/MyDrive/ANN-ProjectData/filtered_dataset_clean.csv"
imagePath = "/content/drive/MyDrive/ANN-ProjectData/ImageData.zip"
imageExtractedPath = "/content/fake_news_images"

with zipfile.ZipFile(imagePath, 'r') as zip_ref:
    zip_ref.extractall(imageExtractedPath)

print("✅ Images extracted to:", imageExtractedPath)


✅ Images extracted to: /content/fake_news_images


In [3]:
import os

# Path to the folder containing images
folder_path = "/content/fake_news_images"  # Change this

# Word or pattern you want to remove
word_to_remove = "image_"

# Go through all files
for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)
    if os.path.isfile(old_path):
        new_filename = filename.replace(word_to_remove, "")
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")


Renamed: image_548.jpg → 548.jpg
Renamed: image_704.jpg → 704.jpg
Renamed: image_4506.jpg → 4506.jpg
Renamed: image_472.jpg → 472.jpg
Renamed: image_1921.jpg → 1921.jpg
Renamed: image_2698.jpg → 2698.jpg
Renamed: image_2755.jpg → 2755.jpg
Renamed: image_6122.jpg → 6122.jpg
Renamed: image_2534.jpg → 2534.jpg
Renamed: image_4378.jpg → 4378.jpg
Renamed: image_5880.jpg → 5880.jpg
Renamed: image_3232.jpg → 3232.jpg
Renamed: image_5164.jpg → 5164.jpg
Renamed: image_6008.jpg → 6008.jpg
Renamed: image_3115.jpg → 3115.jpg
Renamed: image_5986.jpg → 5986.jpg
Renamed: image_464.jpg → 464.jpg
Renamed: image_3087.jpg → 3087.jpg
Renamed: image_4193.jpg → 4193.jpg
Renamed: image_468.jpg → 468.jpg
Renamed: image_789.jpg → 789.jpg
Renamed: image_2128.jpg → 2128.jpg
Renamed: image_4081.jpg → 4081.jpg
Renamed: image_57.jpg → 57.jpg
Renamed: image_6100.jpg → 6100.jpg
Renamed: image_1804.jpg → 1804.jpg
Renamed: image_4973.jpg → 4973.jpg
Renamed: image_5830.jpg → 5830.jpg
Renamed: image_709.jpg → 709.jpg
Ren

In [4]:
class FakeNewsDataset(Dataset):
    def __init__(self, dataframe, image_dir):
        self.df = dataframe
        self.image_dir = image_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, str(row["ID"]) + ".jpg")
        return row["text"], image_path, torch.tensor(row["label"], dtype=torch.long)

def collate_fn(batch):
    texts, image_paths, labels = zip(*batch)
    images = [Image.open(p).convert("RGB") for p in image_paths]
    inputs = processor(text=list(texts), images=images, return_tensors="pt", padding=True, truncation=True)
    return inputs, torch.stack(labels)


In [5]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-base-coco")
blip = BlipModel.from_pretrained("Salesforce/blip-itm-base-coco")
blip.eval()

for param in blip.parameters():
    param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
blip = blip.to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/895M [00:00<?, ?B/s]

`BlipModel` is going to be deprecated in future release, please use `BlipForConditionalGeneration`, `BlipForQuestionAnswering` or `BlipForImageTextRetrieval` depending on your usecase.


model.safetensors:   0%|          | 0.00/895M [00:00<?, ?B/s]

Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-itm-base-coco and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_model.encoder.layer.0.attention.self.value.bias', 'text_model.encoder.layer.0.attention.self.value.weight', 'text_model.encoder.layer.0.crossattention.output.LayerNorm.bias', 'text_model.encoder.

In [6]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=1024):  # 512 for csv file and 512 for the image
        super(MLPClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),

            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.fc(x)


In [7]:
df = pd.read_csv(dataPath)
dataset = FakeNewsDataset(df, imageExtractedPath)
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=8, collate_fn=collate_fn)


In [9]:
model = MLPClassifier(input_dim=1024).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        with torch.no_grad():
            out = blip(**inputs) # take the embedings

            features = torch.cat([out.image_embeds, out.text_embeds], dim=1)  # Shape [B, 1024]

        preds = model(features)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

Epoch 1 Loss: 0.6886
Epoch 2 Loss: 0.6565
Epoch 3 Loss: 0.6558
Epoch 4 Loss: 0.6455
Epoch 5 Loss: 0.6395
Epoch 6 Loss: 0.6267
Epoch 7 Loss: 0.6274
Epoch 8 Loss: 0.6284
Epoch 9 Loss: 0.6216
Epoch 10 Loss: 0.6285
Epoch 11 Loss: 0.6218
Epoch 12 Loss: 0.6220
Epoch 13 Loss: 0.6105
Epoch 14 Loss: 0.6087
Epoch 15 Loss: 0.6125


In [10]:
from sklearn.metrics import f1_score, accuracy_score

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        out = blip(**inputs)
        features = torch.cat([out.image_embeds, out.text_embeds], dim=1)
        preds = model(features).argmax(dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

f1 = f1_score(y_true, y_pred, average="weighted")
accuracy = accuracy_score(y_true, y_pred)

print(f"Weighted F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Weighted F1 Score: 0.5948
Accuracy: 0.6269


In [12]:
from sklearn.metrics import f1_score, accuracy_score

def evaluate_accuracy(loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            out = blip(**inputs)
            features = torch.cat([out.image_embeds, out.text_embeds], dim=1)
            preds = model(features).argmax(dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")
    return acc, f1

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        with torch.no_grad():
            out = blip(**inputs)
            features = torch.cat([out.image_embeds, out.text_embeds], dim=1)

        preds = model(features)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # Evaluate on train and test set
    train_acc, train_f1 = evaluate_accuracy(train_loader)
    test_acc, test_f1 = evaluate_accuracy(test_loader)

    print(
        f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f} | "
        f"Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f} | "
        f"Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}"
    )


Epoch 1/10 - Loss: 0.6103 | Train Acc: 0.6189, Train F1: 0.5594 | Test Acc: 0.5981, Test F1: 0.5386
Epoch 2/10 - Loss: 0.6061 | Train Acc: 0.7380, Train F1: 0.7352 | Test Acc: 0.6671, Test F1: 0.6648
Epoch 3/10 - Loss: 0.6000 | Train Acc: 0.7364, Train F1: 0.7317 | Test Acc: 0.6701, Test F1: 0.6662
Epoch 4/10 - Loss: 0.5966 | Train Acc: 0.6724, Train F1: 0.6690 | Test Acc: 0.6629, Test F1: 0.6590
Epoch 5/10 - Loss: 0.6001 | Train Acc: 0.7280, Train F1: 0.7176 | Test Acc: 0.6671, Test F1: 0.6568
Epoch 6/10 - Loss: 0.6006 | Train Acc: 0.7424, Train F1: 0.7381 | Test Acc: 0.6749, Test F1: 0.6710
Epoch 7/10 - Loss: 0.5852 | Train Acc: 0.6899, Train F1: 0.6884 | Test Acc: 0.6677, Test F1: 0.6652
Epoch 8/10 - Loss: 0.5862 | Train Acc: 0.7418, Train F1: 0.7409 | Test Acc: 0.6773, Test F1: 0.6768
Epoch 9/10 - Loss: 0.5988 | Train Acc: 0.7457, Train F1: 0.7440 | Test Acc: 0.6755, Test F1: 0.6743
Epoch 10/10 - Loss: 0.5867 | Train Acc: 0.7238, Train F1: 0.7238 | Test Acc: 0.6797, Test F1: 0.6795

In [15]:
# Comparision:


